In [1]:
import json, wandb
from importlib import reload

In [2]:
with open('experiments/todo/0GCN.json') as file:
        construct_dict = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/todo/0GCN.json'

In [ ]:
construct_dict['data_params']['n_data']=100
construct_dict['exp_name']='boo'

In [ ]:
construct_dict

In [ ]:
import dev.data_load as dl
reload(dl)
data=dl.graph_data

In [ ]:
data(**construct_dict['data_params'])

In [ ]:
import tensorflow as tf
tf.math.acos(1.1)

In [ ]:
eps=1
tf.math.acos

In [ ]:
%run run_trainings.py

GPU detected
Cleaned done folder
Starting process with 4 experiments
['1GCN.json', '3GCN.json', '3GCN0ECCC.json', '6GCN.json']
Starting experiment from 1GCN


wandb: Currently logged in as: chri862z (use `wandb login --relogin` to force relogin)


Not notebook
Loading data to memory
check
loading model


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

New folder for saving run made


In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)